In [1]:
cd C:\\Users\\Sharuka Thirimanne\\Desktop\\FYP-ML-IDS

C:\Users\Sharuka Thirimanne\Desktop\FYP-ML-IDS


In [3]:
import sklearn
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import Model, layers
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix,f1_score

#Pipelines
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from pickle import dump
from pickle import load
from sklearn.pipeline import Pipeline
from io import StringIO
from keras.models import model_from_json
import joblib


In [4]:
# Load the model
estimator = model_from_json(open('model_architecture.json','r').read())
estimator.load_weights('model_weights.h5')

# Load Column Transformer 
column_transformer = joblib.load('columnTransformer')

In [5]:
METRICS = [ 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
estimator.compile(loss='binary_crossentropy', optimizer='sgd', metrics = METRICS)

In [6]:
pipe = Pipeline(steps=[
    ('preprocessing',column_transformer),
    ('classifier',estimator)])

In [14]:
cc = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes','dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
      'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell','su_attempted', 'num_root', 'num_file_creations', 
      'num_shells','num_access_files', 'num_outbound_cmds', 'is_host_login','is_guest_login', 'count', 'srv_count', 
      'serror_rate','srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate','diff_srv_rate', 'srv_diff_host_rate', 
      'dst_host_count','dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
      'dst_host_srv_diff_host_rate', 'dst_host_serror_rate','dst_host_srv_serror_rate', 'dst_host_rerror_rate',
      'dst_host_srv_rerror_rate']

x2 = "0,tcp,http,SF,267,14515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0,0,0,0,1,0,0,155,255,1,0,0.01,0.03,0.01,0,0,0"
str_data = StringIO(x2)

df_tt = pd.read_csv(str_data,sep=",",names=cc)
df_tt

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,267,14515,0,0,0,0,...,155,255,1,0,0.01,0.03,0.01,0,0,0


In [15]:
test_predictions = pipe.predict(df_tt)
print('Predicted value : ',test_predictions)

if test_predictions>0.5:
    output = 'Anomaly'
else:
    output = 'Normal'
        
print('Status : ',output)

Predicted value :  [[3.9138504e-06]]
Status :  Normal
